In [2]:
# Run this code. Importing easyocr won't work if you don't
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [3]:
import easyocr

In [9]:
# reader = easyocr.Reader(['ch_tra', 'en'])
# result = reader.readtext('chinese_tra.jpg')